In [3]:
import numpy as np
import pandas as pd
from importlib import reload

import munch
import datetime

import shared
import config
import provider_yfinance as provider

reload(shared)
reload(config)
reload(provider)

cfg = config.get_config('^GDAXI')

# overwrite download_end_dt: use cached data
config.overwrite_end_dt(cfg, '2019-12-19')
config.save_config(cfg)

config> read from config file: 'Munch({'base': Munch({'cache_dir': '/mnt/c/notebooks/sandbox/cache', 'cache_enabled': False, 'config_file_path': '/mnt/c/notebooks/sandbox/config.json', 'current_dir': '/mnt/c/notebooks/sandbox', 'model_dir': '/mnt/c/notebooks/sandbox/model'}), 'datasets': Munch({'raw': Munch({'benchmarks': ['000001.SS', 'AUDUSD=X', 'BTCUSD=X', 'CNY=X', 'ETHUSD=X', 'EURCAD=X', 'EURCHF=X', 'EURGBP=X', 'EURHUF=X', 'EURJPY=X', 'EURJPY=X', 'EURSEK=X', 'EURUSD=X', 'GBPJPY=X', 'GBPUSD=X', 'GC=F', 'HKD=X', 'IDR=X', 'IMOEX.ME', 'INR=X', 'JPY=X', 'KW=F', 'LC=F', 'LH=F', 'MXN=X', 'MYR=X', 'NZDUSD=X', 'PA=F', 'PHP=X', 'PL=F', 'RUB=X', 'S=F', 'SGD=X', 'SI=F', 'SM=F', 'THB=X', 'ZAR=X', '^AORD', '^AXJO', '^BFX', '^BSESN', '^BUK100P', '^BVSP', '^DJI', '^FCHI', '^FTSE', '^GDAXI', '^GSPC', '^GSPTSE', '^HSI', '^IPSA', '^IXIC', '^JKSE', '^JN0U.JO', '^KLSE', '^KS11', '^MERV', '^MXX', '^N100', '^N225', '^NYA', '^NZ50', '^RUT', '^STI', '^STOXX50E', '^TA125.TA', '^TWII', '^VIX', '^XAX'], 'stoc

In [ ]:
submodel_settings = cfg.train.settings[0]
print(f"sm-{submodel_settings.id}> training submodel ...")
model_data = provider.prepare_submodel_data(cfg, submodel_settings)


In [ ]:

from keras.preprocessing.sequence import pad_sequences

seq = x.apply(tuple, axis=1).apply(list)
assert len(seq) == submodel_settings.lookback_days, f'len(seq)={len(seq)} must equal lookback_days={submodel_settings.lookback_days}'
pad_sequences([seq.tolist()], submodel_settings.lookback_days, dtype='float32')


In [14]:
X = next(iter(model_data.values())).X
seq = X[-1].apply(tuple, axis=1).apply(list)
pad_sequences([seq.tolist()], submodel_settings.lookback_days, dtype='float32')


array([[[ 2.7479093 ,  3.7275987 ,  2.3416965 , ...,  1.0287318 ,
          1.2776679 ,  0.        ],
        [ 2.8912783 ,  2.8912783 , -0.14336918, ...,  1.1391221 ,
          1.2531807 ,  0.        ],
        [ 0.        ,  1.027479  , -0.45400238, ...,  1.4358435 ,
          1.6249752 ,  0.        ]]], dtype=float32)

In [13]:
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarn